# Cavity Dynamics

This notebook measures the cavity field as a function of time. The cavity is coupled to a transmon qubit. As such the cavity field is measured for the qubit in both the ground and excited states. The pi-pulse used to excite the qubit is obtained from the latest backend calibration. 

## 1. Preliminaries

Connect to the IBM Quantum Experience. 

In [1]:
from qiskit.tools.jupyter import *
from qiskit import IBMQ

import numpy as np
import matplotlib.pyplot as plt

# do this so that when call backend, will look nice, report properties

%config InlineBackend.figure_format = 'svg' # Makes the images look nice

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support OpenPulse"

# check qiskit version
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.14.1',
 'qiskit-aer': '0.5.1',
 'qiskit-ignis': '0.3.0',
 'qiskit-ibmq-provider': '0.7.0',
 'qiskit-aqua': '0.7.0',
 'qiskit': '0.19.1'}

Get the parameters for the specific qubit

In [ ]:
# basic unit conversion factors and device properties

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds
scale_factor = 1e-14 # scale factor to remove factors of 10 from the data

# basic parameters
qubit = 0 # there is only one qubit
dt = backend_config.dt
#defaults
backend_defaults = backend.defaults()

# drive parameters
[[min_drive_freq, max_drive_freq]] = backend_config.qubit_lo_range # range of freqs for qubit drive, 1Ghz around wq
[[min_meas_freq, max_meas_freq]] = backend_config.meas_lo_range # range of freqs for measurement drive

# cavity parameters
cavity_freq = backend_defaults.meas_freq_est[qubit]

# qubit parameters
qubit_props_dict = backend.properties().qubit_property(0)
qubit_freq = qubit_props_dict['frequency'][0]
# qubit_freq = backend_defaults.qubit_freq_est[qubit] #same result
qubit_T1 = qubit_props_dict['T1'][0]
qubit_T2 = qubit_props_dict['T2'][0]

print(f"sampling time: {dt / ns : .2f} ns.")
print(f"qubit frequency: {qubit_freq / GHz} GHz.")
print(f"qubit decay: {1 /(qubit_T1* GHz)} GHz.")
print(f"cavity frequency: {cavity_freq / GHz} GHz.")

print(f"qubit T1: {qubit_T1/us} us.")

In [ ]:
# defaults and other parameters used to define control pulses
from qiskit import pulse
from qiskit.pulse import pulse_lib
from qiskit.pulse import Play # comment out if qiskit-terra vesrion less than 0.13.0

# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)


# Find out which group of qubits need to be acquired with this qubit
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
print(f"Qubit in measurement group {meas_map_idx}")
qubit_meas_group = backend_config.meas_map[meas_map_idx]

#default instructions
inst_sched_map = backend_defaults.instruction_schedule_map #default pulses so dont have to construct by hand

drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)

## 2. Pulse Schedule
- (Optional) Add a pi-pulse to excite the qubit
- Apply a very short measurement pulse and turn on acquisition channel.
- The duration for when the measurement pulse is switched on is varied.
- The acquisition channel is turned on directly after the measurement tone ends.
- This allows us to measure the cavity field operator at various times. 
- The acquisition pulse is of the same amplitude as the measurement pulse.
- There are two sets of schedules, one where the qubit is initially prepared in the excited state via the pi-pulse (derived from the backend), and one where the qubit remains in the ground state.

In [ ]:
# acqSamples_us         : Duration of the acquisition window, in us
# cavityPumpDuration_us : List of times delays (in us) between the start of the measurement and when the cavity field is acquired.
# measAmp               : Strength of the measurement drive
# measSigma_us          : Width of the Gaussian part of the measurement Pulse, in us
# measRiseFall_us       : Rise and fall time of the Gaussian part of the measurement Pulse, in us
def generateSchedules(acqSamples_us, cavityPumpDuration_us, measAmp, measSigma_us, measRiseFall_us):
        
    # convert all the above durations into samples (in units of dt)
    # then quantize the samples to the closest multiple of 16
    cavityPumpSamples = [get_closest_multiple_of_16(cavityPumpTime_us * us/dt) for cavityPumpTime_us in cavityPumpDuration_us]
    acqSamples = get_closest_multiple_of_16(acqSamples_us * us/dt)
    measSigma = get_closest_multiple_of_16(measSigma_us * us/dt)       
    measRiseFall = get_closest_multiple_of_16(measRiseFall_us * us/dt) 

    # get the Pi-pulse from the backend; the pulse already has a channel associated with it
    pi_pulse = inst_sched_map.get('x', qubit_meas_group)
    
    ######################## Acquisiton portion of schedule ########################
    meas_pulse = pulse_lib.gaussian_square(duration=acqSamples, 
                                   sigma=measSigma,
                                   amp=measAmp,
                                   risefall=measRiseFall,
                                   name='measurement_pulse')
    # apply meas_pulse to measurement channel
    measure_schedule = Play(meas_pulse, meas_chan) 
    # this acquire object is how add variable duration acquisition windows turns on acquisition channel
    measure_schedule += pulse.Acquire(duration=acqSamples, 
                                      channel=[pulse.AcquireChannel(i) for i in qubit_meas_group],
                                      mem_slot=[pulse.MemorySlot(i) for i in qubit_meas_group])

    ################ Make the schedule for ground-state preparation ################
    schedules_gnd = []
    
    # iterate through each cavity pump duration
    for cavityPumpSample in cavityPumpSamples: 
        # create an empty schedule
        schedule = pulse.Schedule(name=f"|g> readout dynamics t={cavityPumpSample * dt/us} us")
        
        # construct a measurement pulse with duration cavityPumpSample
        cavity_pulse = pulse_lib.gaussian_square(duration=cavityPumpSample,
                                   sigma=measSigma,
                                   amp=measAmp,
                                   risefall=measRiseFall,
                                   name='cavity_pump_pulse')
        
        # apply the measurement pulse to the measurement channel
        schedule += Play(cavity_pulse, meas_chan)
        
        # add the acquisition schedule (as defined above) immediately after the the measurement tone
        schedule += measure_schedule
        
        # append to the list of schedules for the ground state
        schedules_gnd.append(schedule)


    ############### Make the schedule for excited-state preparation ###############    
    schedules_exc = []
    
    # iterate through each cavity pump duration
    for cavityPumpSample in cavityPumpSamples: 
        # create an empty schedule
        schedule = pulse.Schedule(name=f"|e> readout dynamics t={cavityPumpSample * dt/us} us")
        
        # add the pi-pulse
        schedule += pi_pulse
        
        # construct a measurement pulse with duration cavityPumpSample
        cavity_pulse = pulse_lib.gaussian_square(duration=cavityPumpSample,
                                   sigma=measSigma,
                                   amp=measAmp,
                                   risefall=measRiseFall,
                                   name='cavity_pump_pulse')
        
        # apply the measurement pulse to the measurement channel; delay this to after the pi-pulse is completed
        schedule += Play(cavity_pulse, meas_chan) << schedule.duration
        
        # add the acquisition schedule (as defined above) immediately after the the measurement tone
        schedule += measure_schedule
        
        # append to the list of schedules for the excited state
        schedules_exc.append(schedule)
    
    
    return (schedules_gnd, schedules_exc)

### Assemble and Run the Job

In [ ]:
from qiskit import assemble
from qiskit.tools.monitor import job_monitor


# driveFreq      : Frequency of the Driving (in Hz)
# measFreq       : Frequency of the Measurement (in Hz)
# numShotsPerFreq: Number of experiments per frequency
# numShotsPerFreq: Number of experiments per frequency
# schedules_gnd  : List of ground state schedules
# schedules_exc  : List of excited state schedules
def runJob(driveFreq, measFreq, numShotsPerFreq, schedules_gnd, schedules_exc):
    # Match the frequencies to the channels
    channel_freqs = {drive_chan: driveFreq, meas_chan: measFreq}
    
    # Assemble the jobs 
    acquire_sweep_program_gnd = assemble(schedules_gnd,
                                        backend=backend, 
                                        meas_level=1,
                                        meas_return='avg',
                                        shots=numShotsPerFreq,
                                        schedule_los=[channel_freqs] * len(schedules_gnd))
    acquire_sweep_program_exc = assemble(schedules_exc,
                                        backend=backend, 
                                        meas_level=1,
                                        meas_return='avg',
                                        shots=numShotsPerFreq,
                                        schedule_los=[channel_freqs] * len(schedules_exc))

    # Run the jobs
    job_gnd = backend.run(acquire_sweep_program_gnd)
    print('gnd sweep: ' + job_gnd.job_id())
    
    job_exc = backend.run(acquire_sweep_program_exc)
    print('exc sweep: '+ job_exc.job_id())

    return (job_gnd.job_id(), job_exc.job_id())

## 3. Dynamics of the Cavity Field

In [ ]:
def cavityDynamics(startTime_us, stopTime_us, timeStep_us, acqSamples_us, measAmp, measSigma_us, 
                   measRiseFall_us, driveFreq, measFreq, numShotsPerFreq):
    
    ########### insert code to automatically split
    readout_values_gnd = []
    readout_values_exc = []
    
    cavityPumpTimes_us = np.arange(startTime_us, stopTime_us, timeStep_us)
    print("\nRunning Cavity Dynamics from {0}us to {1}us, in steps of {2}us".format(cavityPumpTimes_us[0], 
                                                                                    cavityPumpTimes_us[-1],
                                                                                    timeStep_us))

    (schedules_gnd, schedules_exc) = generateSchedules(acqSamples_us, cavityPumpTimes_us, 
                                                      measAmp, measSigma_us, measRiseFall_us)
    (gndId, excId) = runJob(driveFreq, measFreq, numShotsPerFreq, 
                            schedules_gnd, schedules_exc)

    job_gnd = backend.retrieve_job(gndId)
    job_exc = backend.retrieve_job(excId)

    print("Monitoring the Ground Sweep: ")
    job_monitor(job_gnd)

    print("\nMonitoring the Excited Sweep: ")
    job_monitor(job_exc)

    print()

    print("Errors for the Ground Sweep: {0}".format(job_gnd.error_message()))
    print("Errors for the Excited Sweep: {0}".format(job_exc.error_message()))

    # timeout parameter set to 120 seconds
    acquire_sweep_gnd_results = job_gnd.result(timeout=120) 
    acquire_sweep_exc_results = job_exc.result(timeout=120)

    # load complex measurement results, scale
    for i in range(len(cavityPumpTimes_us)):
        readout_values_gnd.append(acquire_sweep_gnd_results.get_memory(i)[qubit]*scale_factor)

    for i in range(len(cavityPumpTimes_us)):
        readout_values_exc.append(acquire_sweep_exc_results.get_memory(i)[qubit]*scale_factor)
    
    return (readout_values_gnd, readout_values_exc)

Define the device-specific parameters

In [ ]:
chi = 0.421 * MHz
measFreq = cavity_freq
driveFreq = qubit_freq - chi

Qiskit limits the number of points per job. For shorter times, we can measure the cavity field for a wide window (for example, $0.1 \mu s$ to $3.2 \mu s$). However, the number of points increases with long times, as such we can pack fewer experiments into each job. As such, the width of the time-window shrinks with time. 

The homodyne measurement of the cavity field introduces a spurious phase shift for every job run. As such, we need at least one point to overlap between consecutive time windows. We use this overlap point to rotate the IQ plane of the latter measurement to match the previous measurement. 

The shrinking time-window and the need for an overlap point sets an upper bound to the longest time we can measure up to. This is because, there comes a point after which the time-windows can't accomodate both the overlap point and new measurement points. 

In [ ]:
readout_values_gnd = []
readout_values_exc = []

# A list of the start and stop time pairs for each measurement window
cavityPumpStartStop_us = [ [0.1, 3.2], [3.1, 4.5], [4.4, 5.5], [5.4, 6.0], [5.9, 6.8], [6.7, 7.5], [7.4, 8.1], 
                           [8.0, 8.6], [8.5, 9.1], [9.0, 9.6], [9.5, 10.0], [9.9, 10.4], [10.3, 10.7], [10.6, 11.0],
                           [11.0, 11.5], [11.4, 11.7], [11.6, 11.9], [11.9, 12.2] , [12.1, 12.4], [12.4, 12.7], 
                           [12.6, 12.9], [12.9, 13.2], [13.1, 13.4], [13.4, 13.7], [13.6, 13.9], [13.9, 14.2], 
                           [14.1, 14.4], [14.4, 14.7]]

idx = 0
for [startTime, stopTime] in cavityPumpStartStop_us:
    
    # run the job for the assigned window
    (readout_values_gnd_curr, readout_values_exc_curr) = cavityDynamics(startTime_us = startTime, stopTime_us = stopTime, 
                                                                        timeStep_us = 0.1, acqSamples_us = 0.05, 
                                                                        measAmp = 0.5, measSigma_us = 0.01, 
                                                                        measRiseFall_us = 0.001, driveFreq = driveFreq, 
                                                                        measFreq = measFreq, numShotsPerFreq = 1024)
    
    # rotate the measured values to correct for the phase shift from the homodyne measurement
    # multiply with a phasor; get this phasor based on the points that overlaps between consecutive windows
    if (idx > 0):
        rot_phasor_gnd = readout_values_gnd[-1]/readout_values_gnd_curr[0]
        readout_values_gnd += [ rot_phasor_gnd * val for val in  readout_values_gnd_curr[1:]]

        rot_phasor_exc = readout_values_exc[-1]/readout_values_exc_curr[0]
        readout_values_exc += [ rot_phasor_exc * val for val in  readout_values_exc_curr[1:]]
    
    else:
        readout_values_gnd = readout_values_gnd_curr
        readout_values_exc = readout_values_exc_curr
        
    # keep count    
    idx += 1

Rotate the entire plane such that the ground state measurement is entirely in the Q quadrature.

In [ ]:
rot_phasor = np.average(readout_values_gnd)/np.absolute(np.average(readout_values_gnd))
rot_phasor = np.conj(rot_phasor)

readout_values_gnd_rot =  [rot_phasor * val for val in  readout_values_gnd]
readout_values_exc_rot =  [rot_phasor * val for val in  readout_values_exc]

In [ ]:
measAmp = 0.5
duration_us = np.arange(0.1, 14.7, 0.1)
   
# plot results in figure
fig =plt.figure(figsize=(8,5))
fig.suptitle(f"Dispersive single-qubit readout dynamics - Rotated, $\epsilon_m$ = {measAmp}, $\omega_d = {driveFreq/GHz}$ GHz, $\omega_m = {measFreq/GHz} GHz$")
ax_real=fig.add_axes([.1, .2, .4, .7])
ax_imag=fig.add_axes([.6, .2, .4, .7])

ax_real.plot(duration_us, np.real(readout_values_gnd_rot), '-ob')
ax_real.plot(duration_us, np.real(readout_values_exc_rot), '-or')

ax_imag.plot(duration_us, np.absolute(np.imag(readout_values_gnd_rot)), '-ob')
ax_imag.plot(duration_us, np.absolute(np.imag(readout_values_exc_rot)), '-or')

ax_real.set_xlabel('Measurement time after $\pi$ pulse [$\mu$s]')
ax_imag.set_xlabel('Measurement time after $\pi$ pulse [$\mu$s]')
ax_real.set_ylabel('Measured Q [a.u.]')
ax_imag.set_ylabel('Measured I [a.u.]')
ax_real.legend(['$\\vert g\\rangle$', '$\\vert e\\rangle$'])

## 4. Save the Data

In [ ]:
import csv
import sys

fname = "measAmp{:0.2f}_driveFreq{:0.2f}GHz_measFreq{:0.2f}GHz.csv".format(measAmp, driveFreq/GHz, measFreq/GHz)
with open(fname, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['time(us)', 'GND_I', 'GND_Q', 'EXC_I', 'EXC_Q', 'GND_rot_I', 'GND_rot_Q', 'EXC_rot_I', 'EXC_rot_Q'])
     
    times = np.arange(0.1, 5.0, 0.25)
    for i in range(len(times)):
        writer.writerow([times[i],
                         np.real(readout_values_gnd_curr[i]), np.imag(readout_values_gnd_curr[i]),
                         np.real(readout_values_exc_curr[i]), np.imag(readout_values_exc_curr[i]), 
                         np.real(readout_values_gnd_rot[i]), np.imag(readout_values_gnd_rot[i]), 
                         np.real(readout_values_exc_rot[i]), np.real(readout_values_exc_rot[i])])